In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
import os

from taogpt.orchestrator import *
from taogpt.utils import *
from taogpt.llm_model import LangChainLLM
from taogpt.prompts import PromptDb
import taogpt.utils as utils

utils.enable_debugging(0)

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
TEMPERATURE = 0.7

In [4]:
with open(os.path.join(os.environ['HOME'], '.ssh', 'openai-zillow.json'), 'r') as f:
    credentials = json.load(f)
os.environ["OPENAI_API_KEY"] = credentials['key']
os.environ["OPENAI_API_BASE"] = credentials['url']
llm3_5 = ChatOpenAI(model_name='gpt-3.5-turbo-16k', temperature=TEMPERATURE)
llm4 = ChatOpenAI(model_name='gpt-4-32k', temperature=TEMPERATURE)

conversation = ConversationChain(llm=llm3_5)
conversation.predict(input="What's your model version?")

"I am using OpenAI's GPT-3 model, which is the third iteration of the Generative Pre-trained Transformer. It was released in June 2020 and has been trained on a wide variety of internet text to generate human-like responses."

In [5]:
llm3_5.model_name, llm4.model_name

('gpt-3.5-turbo-16k', 'gpt-4-32k')

In [6]:
prompts = PromptDb.load_defaults()
logger = MarkdownLogger('logs/taogpt_log.md')
config = Config(
    ask_user_before_execute_codes=False,
    ask_user_questions_in_one_prompt=True,
    pause_after_initial_solving_expansion=True,
    first_expansion=1,
    initial_expansion=4,
    max_tree_branches=6,
    max_tokens=5000,
    check_final=True
)
executor = Orchestrator(
    config=config,
    llm=LangChainLLM(llm4, logger=logger),
    prompts=prompts,
    markdown_logger=logger,
    # sage_llm=LangChainLLM(llm4, logger=logger),
)

experiment_name = 'example'
executor.start("""
Solve this 4x4 Sudoku represented by the 2-D Python array where 0 is the empty cell:

```python
[
    [0, 3, 0, 1],
    [1, 0, 0, 3],
    [2, 0, 3, 4],
    [3, 4, 0, 2],
]
```

Solve it in your head, do not write codes.
""", analyze_first=True)

Pause: Pause after initial solving expansion

In [7]:
executor.resume(10000, additional_tokens_for_smarter_llm=1000, unblock_initial_expansion=True)

extend token allowance by 10000 to 15000
extend token allowance for smarter LLM by 1000 to 6000


RuntimeError: Regular LLM consumed 17042 tokens, exceeded allowance of 15000

In [42]:
logger = MarkdownLogger(f'examples/{experiment_name}.final.md')
logger.log_conversation(executor.show_conversation_thread(with_extras=True))
logger.log(f"**total tokens**: {executor.llm.total_tokens}")
